In [ ]:
import sys
import os
import math
import logging
import random
import torch
import traceback
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.cuda.amp import GradScaler, autocast
import pandas as pd
import requests
import io
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from typing import List, Tuple, Optional, Dict, Any
# パフォーマンス設定
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cudnn.benchmark = True
torch.set_float32_matmul_precision('high')

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ========================================
# データセット定義
# ========================================
class EnhancedLoto7Dataset(Dataset):
    def __init__(self, csv_url, window_size=10):
        self.window_size = window_size

        # データ取得と前処理
        response = requests.get(csv_url)
        response.encoding = 'shift_jis'
        df = pd.read_csv(io.StringIO(response.text))

        # 数字を整数として取得（1～37を0～36に変換）
        raw_draws = df[['第1数字','第2数字','第3数字','第4数字','第5数字','第6数字','第7数字']].values.astype(np.int64)
        self.draws_int = raw_draws - 1  # ターゲット（0～36）
        # 入力用は正規化（0～1）
        self.draws_norm = (raw_draws - 1) / 36.0

        # 開催回の正規化：各回の開催回を1次元の数値として扱う
        self.draw_numbers = df['開催回'].values
        self.min_draw_number = self.draw_numbers.min()
        self.max_draw_number = self.draw_numbers.max()-2
        self.draw_numbers_norm = (self.draw_numbers - self.min_draw_number) / (self.max_draw_number - self.min_draw_number)

        # サンプル生成
        self.samples = []
        for i in range(self.window_size, len(self.draws_int)):
            src = np.concatenate([
                self.draw_numbers_norm[i-self.window_size:i].reshape(-1, 1),
                self.draws_norm[i-self.window_size:i]
            ], axis=1).flatten()
            tgt = self.draws_int[i]
            self.samples.append((torch.FloatTensor(src), torch.LongTensor(tgt)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

    def get_recent_data_for_prediction(self, draw_number, models=None):
        """
        指定された開催回に基づいて、過去のデータをウィンドウサイズ分取得する
        未来の開催回の場合は、再帰的に予測してデータを補充する
        """
        # 開催回がデータセット内にある場合
        if draw_number in self.draw_numbers:
            idx = np.where(self.draw_numbers == draw_number)[0][0]
            if idx < self.window_size:
                raise ValueError(f"指定された開催回 {draw_number} のインデックス {idx} はウィンドウサイズ {self.window_size} よりも小さいため、過去データが不足しています。")
            
            src = np.concatenate([
                self.draw_numbers_norm[idx-self.window_size:idx].reshape(-1, 1),
                self.draws_norm[idx-self.window_size:idx]
            ], axis=1).flatten()
            return torch.FloatTensor(src)
        
        # 未来の開催回の場合
        elif draw_number > self.max_draw_number:
            if models is None:
                raise ValueError("未来の開催回を予測するにはモデルが必要です")
            
            # 最新のデータを取得
            latest_idx = len(self.draw_numbers) - 1
            latest_draw_number = self.draw_numbers[latest_idx]
            
            # 必要な未来のステップ数を計算
            steps = draw_number - latest_draw_number
            
            # 最新のウィンドウデータを取得
            window_data = np.concatenate([
                self.draw_numbers_norm[latest_idx-self.window_size+1:latest_idx+1].reshape(-1, 1),
                self.draws_norm[latest_idx-self.window_size+1:latest_idx+1]
            ], axis=1)
            
            # 未来のデータを予測して補充
            for _ in range(steps):
                # 入力データを準備 (正規化)
                input_data = window_data[-self.window_size:].flatten()
                input_tensor = torch.FloatTensor(input_data).unsqueeze(0)
                
                # モデルで予測
                predicted_draws = self.predict_next_draw(input_tensor, models)
                
                # 新しい開催回を作成 (最新の開催回 +1)
                new_draw_number = latest_draw_number + 1
                latest_draw_number = new_draw_number
                
                # 新しいデータポイントを作成 (正規化)
                new_draw_number_norm = (new_draw_number - self.min_draw_number) / (self.max_draw_number - self.min_draw_number)
                new_draws_norm = predicted_draws / 36.0
                
                # ウィンドウデータに追加
                new_data_point = np.concatenate([
                    np.array([new_draw_number_norm]).reshape(1, 1),
                    new_draws_norm.reshape(1, 7)
                ], axis=1)
                
                window_data = np.concatenate([window_data, new_data_point])
            
            # 最後のwindow_size分を返す
            src = window_data[-self.window_size:].flatten()
            return torch.FloatTensor(src)
        
        else:
            raise ValueError(f"指定された開催回 {draw_number} は不正な値です。")

    def predict_next_draw(self, input_tensor, models, num_samples=5):
        """
        入力データから次の抽選結果を予測する
        """
        device = next(models[0].parameters()).device
        input_tensor = input_tensor.to(device)
        
        all_preds = []
        for model in models:
            model.eval()
            with torch.inference_mode(), torch.cuda.amp.autocast():
                logits = model(input_tensor)  # (1, 7, 37)
                probs = F.softmax(logits, dim=-1)[0]  # (7, 37)
                
                # 各位置ごとに確率に基づいてサンプリング
                preds = []
                for pos in range(7):
                    for _ in range(3):  # リトライ機構
                        idx = torch.multinomial(probs[pos], 1).item()
                        if 0 <= idx < 37 and idx not in preds:
                            preds.append(idx)
                            break
                    else:
                        preds.append(random.randint(0, 36))
                all_preds.append(preds)
        
        # アンサンブル平均 (最も多く予測された数字を選択)
        final_pred = []
        for pos in range(7):
            pos_preds = [pred[pos] for pred in all_preds]
            # 最頻値を取得 (同数の場合はランダムに選択)
            counts = np.bincount(pos_preds)
            max_count = np.max(counts)
            modes = np.where(counts == max_count)[0]
            final_pred.append(np.random.choice(modes))
        
        return np.array(final_pred)

# ========================================
# ロータリ位置エンベディング関連の関数
# ========================================
class EnhancedRotaryEmbedding(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
        inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2).float() / dim))
        self.register_buffer("inv_freq", inv_freq, persistent=False)

    def forward(self, x):
        seq_len = x.size(1)
        t = torch.arange(seq_len, device=x.device).type_as(self.inv_freq)
        freqs = torch.einsum('i,j->ij', t, self.inv_freq)
        return torch.cat([freqs, freqs], dim=-1)

def rotate_half(x):
    x1, x2 = x.chunk(2, dim=-1)
    return torch.cat((-x2, x1), dim=-1)

def apply_rotary_pos_emb(pos, t):
    rotate_dim = t.size(-1)
    pos = pos[..., :rotate_dim]
    t_rot, t_pass = t[..., :rotate_dim], t[..., rotate_dim:]
    t_rot = t_rot * pos.cos() + rotate_half(t_rot) * pos.sin()
    return torch.cat((t_rot, t_pass), dim=-1)

# ========================================
# Transformerエンコーダブロック
# ========================================
class EnhancedTransformerEncoder(nn.Module):
    def __init__(self, d_model=128, nhead=8, ff_dim=512, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.linear1 = nn.Linear(d_model, ff_dim)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(ff_dim, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src, src_mask=None, src_key_padding_mask=None):
        src2 = self.self_attn(src, src, src, attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(F.gelu(self.linear1(src))))
        src = src + self.dropout2(src2)
        return self.norm2(src)

# ========================================
# Loto7 Transformer モデル定義
# ========================================
class EnhancedLoto7Transformer(nn.Module):
    def __init__(self, window_size=10, d_model=128, nhead=8, num_layers=6, num_classes=37):
        super().__init__()
        self.window_size = window_size
        self.input_dim = 8  # 各時刻の入力は開催回1次元 + 宝くじ数字7次元

        # 開催回と宝くじ数字をそれぞれ別個に埋め込む
        self.draw_num_embed = nn.Linear(1, d_model // 4)
        self.lottery_embed = nn.Linear(7, d_model - d_model // 4)
        self.combined_proj = nn.Linear(d_model, d_model)

        self.rotary = EnhancedRotaryEmbedding(d_model)
        self.encoder_layers = nn.ModuleList([
            EnhancedTransformerEncoder(d_model, nhead) for _ in range(num_layers)
        ])
        self.fc_out = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.GELU(),
            nn.Linear(d_model, num_classes)
        )
        self.dropout = nn.Dropout(0.1)
        self._init_weights()

    def _init_weights(self):
         for p in self.parameters():
             if p.dim() > 1:
                 nn.init.xavier_uniform_(p)

    def forward(self, src):
         src = src.view(-1, self.window_size, self.input_dim)
         draw_num = src[:, :, :1]
         lottery_nums = src[:, :, 1:]
         draw_emb = self.draw_num_embed(draw_num)
         lottery_emb = self.lottery_embed(lottery_nums)
         combined = torch.cat([draw_emb, lottery_emb], dim=-1)
         combined = self.combined_proj(combined)

         pos = self.rotary(combined)
         combined = apply_rotary_pos_emb(pos, combined)
         combined = self.dropout(combined)

         for layer in self.encoder_layers:
              combined = layer(combined)

         return self.fc_out(combined[:, -7:])

# ========================================
# トレーナー定義
# ========================================
# ========================================
# トレーナー定義 (更新版)
# ========================================
class AdvancedTrainer:
    def __init__(self, model, dataset, target_draw_numbers, batch_size=256, grad_accum=2, 
                 continue_mode=False, checkpoint_path=None):
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device)
        self.dataset = dataset
        self.target_draw_numbers = target_draw_numbers
        self.continue_mode = continue_mode
        self.checkpoint_path = checkpoint_path
        self.start_epoch = 1  # デフォルトの開始エポック

        # オプティマイザーとスケジューラーの初期化
        self.optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=2e-4,
            betas=(0.9, 0.98),
            weight_decay=1e-4
        )
        self.scheduler = CosineAnnealingWarmRestarts(self.optimizer, T_0=10, T_mult=2)
        self.scaler = GradScaler(enabled=True)
        self.loss_fn = nn.CrossEntropyLoss()
        self.batch_size = batch_size
        self.grad_accum = grad_accum
        
        # 結果記録用
        self.results = {draw_number: [] for draw_number in target_draw_numbers}
        self.hist_loss = []
        self.hist_lr = []
        
        # チェックポイントディレクトリ作成
        os.makedirs("checkpoints", exist_ok=True)
        os.makedirs("results", exist_ok=True)
        
        # continue_modeがTrueの場合はチェックポイントを読み込む
        if self.continue_mode:
            self._load_checkpoint()

    def _load_checkpoint(self):
        """チェックポイントからモデル状態を復元"""
        if self.checkpoint_path is None:
            # デフォルトのチェックポイントパス
            self.checkpoint_path = "checkpoints/latest_checkpoint.pth"
        
        try:
            logger.info(f"Loading checkpoint from {self.checkpoint_path}")
            checkpoint = torch.load(self.checkpoint_path, map_location=self.device)
            
            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            self.scaler.load_state_dict(checkpoint['scaler_state_dict'])
            
            self.start_epoch = checkpoint['epoch'] + 1
            self.hist_loss = checkpoint['hist_loss']
            self.hist_lr = checkpoint['hist_lr']
            
            # 結果データの読み込み
            for draw_number in self.target_draw_numbers:
                result_file = f"results/prediction_{draw_number}.csv"
                if os.path.exists(result_file):
                    df = pd.read_csv(result_file)
                    self.results[draw_number] = df.to_dict('records')
            
            logger.info(f"Successfully loaded checkpoint. Resuming from epoch {self.start_epoch}")
            return True
        except Exception as e:
            logger.error(f"Failed to load checkpoint: {str(e)}")
            logger.error(traceback.format_exc())
            self.start_epoch = 1
            return False

    def _save_checkpoint(self, epoch):
        """現在の状態をチェックポイントとして保存"""
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'scaler_state_dict': self.scaler.state_dict(),
            'hist_loss': self.hist_loss,
            'hist_lr': self.hist_lr,
        }
        
        torch.save(checkpoint, "checkpoints/latest_checkpoint.pth")
        if epoch %300 == 0:
            torch.save(checkpoint, f"checkpoints/checkpoint_epoch_{epoch}.pth")
            logger.info(f"Checkpoint saved at epoch {epoch}")

    def train_epoch(self, epoch, loader):
        self.model.train()
        total_loss = 0
        self.optimizer.zero_grad()

        for i, (src, tgt) in enumerate(tqdm(loader, desc=f"Epoch {epoch}")):
            src = src.to(self.device, non_blocking=True)
            tgt = tgt.to(self.device, non_blocking=True)

            with autocast(dtype=torch.bfloat16):
                outputs = self.model(src)
                loss = self.loss_fn(outputs.view(-1, outputs.shape[-1]), tgt.view(-1)) / self.grad_accum

            self.scaler.scale(loss).backward()

            if (i+1) % self.grad_accum == 0:
                self.scaler.step(self.optimizer)
                self.scaler.update()
                self.optimizer.zero_grad()
                self.scheduler.step(epoch + i / len(loader))

            total_loss += loss.item()

        avg_loss = total_loss / len(loader)
        self.hist_loss.append(avg_loss)
        current_lr = self.scheduler.get_last_lr()[0]
        self.hist_lr.append(current_lr)
        
        # 予測を実行して結果を記録
        self.record_predictions(epoch, avg_loss, current_lr)
        
        # チェックポイント保存
        self._save_checkpoint(epoch)
        
        logger.info(f"Epoch {epoch} | Loss: {avg_loss:.4f} | LR: {current_lr:.2e}")
    def record_predictions(self, epoch, train_loss, lr):
        """各エポックの予測結果を記録（開催回ごとに個別のCSVに保存）"""
        self.model.eval()
        
        for draw_number in self.target_draw_numbers:
            try:
                # データ取得 (未来の場合は予測で補充)
                recent_src = self.dataset.get_recent_data_for_prediction(
                    draw_number, models=[self.model]).unsqueeze(0).to(self.device)
                
                with torch.inference_mode(), torch.cuda.amp.autocast():
                    logits = self.model(recent_src)
                    probs = F.softmax(logits, dim=-1)
                    
                    # 予測数字を生成 (重複なし)
                    predicted_numbers = []
                    for pos in range(7):
                        for _ in range(3):  # リトライ機構
                            idx = torch.multinomial(probs[0, pos], 1).item()
                            if 1 <= idx+1 <= 37 and idx+1 not in predicted_numbers:
                                predicted_numbers.append(idx+1)
                                break
                        else:
                            predicted_numbers.append(random.randint(1, 37))
                    
                    # 結果を保存
                    result = {
                        'epoch': epoch,
                        'predicted_numbers': ','.join(map(str, predicted_numbers[:7])),
                        'train_loss': train_loss,
                        'lr': lr,
                        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    }
                    self.results[draw_number].append(result)
                    
                    # 開催回ごとにCSVに保存
                    df = pd.DataFrame(self.results[draw_number])
                    df.to_csv(f"results/prediction_{draw_number}.csv", index=False)
            
            except Exception as e:
                logger.error(f"Error predicting for draw {draw_number} at epoch {epoch}: {str(e)}")
# ========================================
# アンサンブル予測生成と可視化
# ========================================
def visualize_probabilities(prob):
    fig, axs = plt.subplots(7, 1, figsize=(10, 20))
    for i in range(7):
        axs[i].bar(range(1, 38), prob[i].detach().cpu().numpy())
        axs[i].set_title(f"Position {i+1} Probability Distribution")
        axs[i].set_xlabel("Digit")
        axs[i].set_ylabel("Probability")
    plt.tight_layout()
    plt.show()

def generate_ensemble_predictions(models, dataset, draw_number, num_preds=5, visualize=False):
    predictions = []
    probs_list = []

    # 未来の開催回の場合、データセットが自動的に予測で補充する
    recent_src = dataset.get_recent_data_for_prediction(draw_number, models=models).unsqueeze(0)
    
    for model in models:
        model.eval()
        device = next(model.parameters()).device
        with torch.inference_mode(), torch.cuda.amp.autocast():
            logits = model(recent_src.to(device))
            probs = F.softmax(logits, dim=-1)
            probs_list.append(probs.cpu())

    avg_probs = torch.stack(probs_list).mean(dim=0)

    if visualize:
        visualize_probabilities(avg_probs[0])

    for _ in range(num_preds):
        nums = []
        for pos in range(7):
            for _ in range(3):
                idx = torch.multinomial(avg_probs[0, pos], 1).item()
                if 1 <= idx+1 <= 37 and idx+1 not in nums:
                    nums.append(idx+1)
                    break
            else:
                nums.append(random.randint(1, 37))
        predictions.append(nums[:7])

    return predictions

# ========================================
# メイン処理
# ========================================
if __name__ == "__main__":
    # データセット初期化
    dataset = EnhancedLoto7Dataset("https://loto7.thekyo.jp/data/loto7.csv")

    # 予測対象の開催回を指定 (過去と未来の開催回を混在させることが可能)
target_draw_numbers = range(624, 634)

# データローダー
loader = DataLoader(
    dataset,
    batch_size=256,
    shuffle=True,
    num_workers=os.cpu_count(),
    pin_memory=True,
    persistent_workers=True
)

# モデルとトレーナーの作成 (continue_modeをTrueに設定してチェックポイントから再開)
continue_mode = True  # コード上でcontinue modeを指定
checkpoint_path = "checkpoints/latest_checkpoint.pth"  # チェックポイントパス

models = [EnhancedLoto7Transformer(window_size=10) for _ in range(1)]
trainers = [
    AdvancedTrainer(
        model, 
        dataset, 
        target_draw_numbers,
        continue_mode=continue_mode,
        checkpoint_path=checkpoint_path
    ) for model in models
]

# トレーニング実行 (start_epochから開始)
start_epoch = trainers[0].start_epoch
end_epoch = 30001

for epoch in range(start_epoch, end_epoch):
    for trainer in trainers:
        trainer.train_epoch(epoch, loader)

# 最終的なアンサンブル予測
final_draw_number = int(input("予測したい開催回を入力してください: "))
predictions = generate_ensemble_predictions(models, dataset, final_draw_number, visualize=True)
pd.DataFrame(predictions).to_csv("enhanced_predictions.csv", index=False)
logger.info("Ensemble prediction results saved with visualization")

/tmp/ipykernel_113950/1051176120.py:300: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler(enabled=True)
/home/codespace/.local/lib/python3.12/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
INFO:__main__:Loading checkpoint from checkpoints/latest_checkpoint.pth
INFO:__main__:Successfully loaded checkpoint. Resuming from epoch 161
Epoch 161:   0%|          | 0/3 [00:00<?, ?it/s]/tmp/ipykernel_113950/1051176120.py:378: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(dtype=torch.bfloat16):
/home/codespace/.local/lib/python3.12/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Epoch 161: 100%|█████